## Imports

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd 
import os 
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### Introducing Selenium 

In [2]:
#The 6th: http://www.santaanita.com/live-racing/4618/ 
##Race 1 & 5


## Horses of Interest

In [46]:
mainpage = "https://www.equibase.com/static/chart/summary/SA040618USA-EQB.html"
horselink = BeautifulSoup(requests.get(mainpage).text, "lxml")


In [47]:
###New list of horses!! Race 1 and 5.
horsenames = ['Dhafeera','Jazaalah','Meal Ticket','You Missed It','Blossom Trail Miss','Chalky','Beauty Divine','Subic Bay',
             'Oh Scatty Oh','Road Test','Hotsy Dotsy','Paddy Jean','Khaleesi','Twirling Apples',
             'Lil Bit Dangerous','Pirate Flag']

In [49]:
horselinkdic = {}

for horse in horsenames:
    for a in horselink.find_all("a", text=re.compile(horse)):
        for i in a:
            a = a['href']
            horselinkdic[horse]=a
            

In [7]:
print((horselinkdic))

{'Dhafeera': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9684715&registry=T&rbt=TB', 'Jazaalah': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9700611&registry=T&rbt=TB', 'Meal Ticket': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9676708&registry=T&rbt=TB', 'You Missed It': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9645455&registry=T&rbt=TB', 'Blossom Trail Miss': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9663458&registry=T&rbt=TB', 'Chalky': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9819822&registry=T&rbt=TB', 'Beauty Divine': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9665823&registry=T&rbt=TB', 'Subic Bay': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9477714&registry=T&rbt=TB', 'Oh Scatty Oh': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9698092&registry=T&rbt=TB', 'Road Test': 'http://www.equibase.com/profiles

## Scraping horsey stats

##### Scraping the stats table

In [8]:
#launch url
chromedriver = "/Users/vicky/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [12]:
horseywinnings=[]
horseyworkouts=[]

for key, value in horselinkdic.items():   
    
    print(key) #So when it inevitably glitches in the middle I know which horse is giving me trouble. 
    jesustakethewheel = webdriver.Chrome(chromedriver)
    jesustakethewheel.get(value)
    
    soupresults=BeautifulSoup(requests.get(value).text, "lxml")
    resultstable=soupresults.find_all(class_='table-hover')[1]

    rows=[row for row in resultstable.find_all('tr')]
    rows=rows[1:5]


    horseywins={}
    for row in rows:
        items=row.find_all('td')
        for entries in items:
            splitsies = entries.text.split(':')
            horseywins[splitsies[0]] = (splitsies[1])
            horseywins['Horse Name'] = (key)    
    
    
    horseywinnings.append(horseywins)
    
    time.sleep(1)
    
    python_button_Workouts = jesustakethewheel.find_element_by_id('Hworkouts') 
    python_button_Workouts.click() 
    time.sleep(.5)
    
    python_button_seemore = jesustakethewheel.find_element_by_link_text('SEE MORE WORKOUTS')
    python_button_seemore.click()
    time.sleep(.5)
    
    soupworkout=BeautifulSoup(jesustakethewheel.page_source, "lxml")
    
    workouttable=soupworkout.find(class_='resultTable') 

    #horseyworkouts=[]
    for row in workouttable.find_all('tr')[1:]:
        items=row.find_all('td')
        Track = items[0].text
        Date = items[1].text
        Course = items[2].text
        Distance = items[3].text
        if len(items[4].text) <6:
            tp = datetime.datetime.strptime(items[4].text,'%S.%f')
        else:
            tp = datetime.datetime.strptime(items[4].text,'%M:%S.%f')
        Time = tp.second*10+tp.minute*600+tp.microsecond//100000
        Note = items[5].text
        Rank = items[6].text
        rowdict= {'Horse Name': key,'Course' :Course,'Track' :Track,'Date':Date ,'Course' :Course ,'Distance' :Distance,'Time_tenths_second':Time,'Note':Note,'Rank':Rank}
        horseyworkouts.append(rowdict)
            
    time.sleep(1)

Dhafeera
Jazaalah
Meal Ticket
You Missed It
Blossom Trail Miss
Chalky
Beauty Divine
Subic Bay
Oh Scatty Oh
Road Test
Hotsy Dotsy
Paddy Jean
Khaleesi
Twirling Apples
Lil Bit Dangerous
Pirate Flag


In [14]:
print(horseywinnings)

[{'Starts': ' 6', 'Horse Name': 'Dhafeera', 'Firsts': ' 1', 'Seconds': ' 2', 'Thirds': ' 0'}, {'Starts': ' 5', 'Horse Name': 'Jazaalah', 'Firsts': ' 0', 'Seconds': ' 1', 'Thirds': ' 2'}, {'Starts': ' 7', 'Horse Name': 'Meal Ticket', 'Firsts': ' 0', 'Seconds': ' 2', 'Thirds': ' 1'}, {'Starts': ' 8', 'Horse Name': 'You Missed It', 'Firsts': ' 0', 'Seconds': ' 5', 'Thirds': ' 1'}, {'Starts': ' 4', 'Horse Name': 'Blossom Trail Miss', 'Firsts': ' 0', 'Seconds': ' 0', 'Thirds': ' 1'}, {'Starts': ' 6', 'Horse Name': 'Chalky', 'Firsts': ' 0', 'Seconds': ' 2', 'Thirds': ' 1'}, {'Starts': ' 4', 'Horse Name': 'Beauty Divine', 'Firsts': ' 0', 'Seconds': ' 0', 'Thirds': ' 0'}, {'Starts': ' 10', 'Horse Name': 'Subic Bay', 'Firsts': ' 2', 'Seconds': ' 3', 'Thirds': ' 1'}, {'Starts': ' 15', 'Horse Name': 'Oh Scatty Oh', 'Firsts': ' 1', 'Seconds': ' 4', 'Thirds': ' 1'}, {'Starts': ' 16', 'Horse Name': 'Road Test', 'Firsts': ' 1', 'Seconds': ' 5', 'Thirds': ' 4'}, {'Starts': ' 19', 'Horse Name': 'Hotsy 

In [15]:
df_stats3 = pd.DataFrame(horseywinnings)
dfworkouts3 = pd.DataFrame(horseyworkouts)

In [16]:
df_stats3.to_csv('stats_csv_3')
dfworkouts3.to_csv('workouts_csv_3')

## Cleaning up dfworkouts

In [3]:
dfworkouts3 = pd.read_csv('workouts_csv_3')

In [4]:
dfworkouts3['Course'].unique()

array(['Dirt', 'Dirt training', 'All Weather Training',
       'All Weather Track', 'Turf training'], dtype=object)

In [6]:
dfworkouts_clean3 = dfworkouts3.drop('Note', axis=1 ).drop('Rank',axis=1).drop('Track',axis=1)
#new_df = df[df.C != 'XYZ']
#dfworkouts_clean = dfworkouts_clean.drop(dfworkouts_clean.loc[dfworkouts_clean['Course'] == 'Turf'])

In [9]:
dfworkouts_clean3.head(2)

,Unnamed: 0,Course,Date,Distance,Horse Name,Time_tenths_second
0,0,Dirt,04/21/2018,4F,Dhafeera,494
1,1,Dirt,04/02/2018,3F,Dhafeera,370


### GROUP BY TO AVERAGE

In [ ]:
dfworkouts_clean.groupby

## Cleaning up df_stats

In [ ]:
df_stats

## Race times and gender df

In [ ]:
resultsofinterest = ['{Horse Name:  , Final_Time}',
                    'http://tnetwork.trakus.com/tnet/t_SantaAnita.aspx?EventID=133974&Date=4/14/2018&Type=TBRED&Venue=4&DisplayType=1',
                    'http://tnetwork.trakus.com/tnet/t_SantaAnita.aspx?EventID=133976&Date=4/14/2018&Type=TBRED&Venue=4&DisplayType=1']

In [23]:
timesdic = [{'Horse Name' : 'Dhafeera', 'Final_Time':'1:40.25', 'Gender':'1' },
           {'Horse Name' :'Jazaalah','Final_Time': '1:40.46', 'Gender':'1'},
           {'Horse Name' :'Meal Ticket','Final_Time': '1:40.58','Gender':'1'},
           {'Horse Name' :'You Missed It','Final_Time': '1:40.84','Gender':'1'},
           {'Horse Name' :'Blossom Trail Miss','Final_Time':'1:40.89','Gender':'1'},
           {'Horse Name' :'Chalky','Final_Time': '1:40.99','Gender':'1'},
           {'Horse Name' :'Beauty Divine','Final_Time': '1:41.75','Gender':'1'},
           {'Horse Name' :'Subic Bay','Final_Time': '1:35.18','Gender':'1'},
           {'Horse Name' :'Oh Scatty Oh','Final_Time': '1:35.59','Gender':'1'},
           {'Horse Name' :'Road Test','Final_Time': '1:36.12','Gender':'1'},
           {'Horse Name' :'Hotsy Dotsy','Final_Time':'1:36.22','Gender':'1'},
           {'Horse Name' :'Paddy Jean','Final_Time': '1:36.24','Gender':'1'},
           {'Horse Name' :'Khaleesi','Final_Time': '1:36.35','Gender':'1'},
           {'Horse Name' :'Twirling Apples','Final_Time':'1:36.67','Gender':'1'},
           {'Horse Name' :'Lil Bit Dangerous','Final_Time': '1:36.74','Gender':'1'},
           {'Horse Name' :'Pirate Flag','Final_Time':'1:37.27','Gender':'1'}]


In [24]:
##This is right for hundreths??? Double check what you scraped!!!

for row in timesdic:
    for k,v in row.items():
        if k == 'Final_Time':
            timing = datetime.datetime.strptime(v,'%M:%S.%f')
            value = timing.second*100+timing.minute*6000+timing.microsecond//10000
        
    
    row['Final_Time_Hund'] = value 

print(timesdic)
    
    

[{'Horse Name': 'Dhafeera', 'Final_Time': '1:40.25', 'Gender': '1', 'Final_Time_Hund': 10025}, {'Horse Name': 'Jazaalah', 'Final_Time': '1:40.46', 'Gender': '1', 'Final_Time_Hund': 10046}, {'Horse Name': 'Meal Ticket', 'Final_Time': '1:40.58', 'Gender': '1', 'Final_Time_Hund': 10058}, {'Horse Name': 'You Missed It', 'Final_Time': '1:40.84', 'Gender': '1', 'Final_Time_Hund': 10084}, {'Horse Name': 'Blossom Trail Miss', 'Final_Time': '1:40.89', 'Gender': '1', 'Final_Time_Hund': 10089}, {'Horse Name': 'Chalky', 'Final_Time': '1:40.99', 'Gender': '1', 'Final_Time_Hund': 10099}, {'Horse Name': 'Beauty Divine', 'Final_Time': '1:41.75', 'Gender': '1', 'Final_Time_Hund': 10175}, {'Horse Name': 'Subic Bay', 'Final_Time': '1:35.18', 'Gender': '1', 'Final_Time_Hund': 9518}, {'Horse Name': 'Oh Scatty Oh', 'Final_Time': '1:35.59', 'Gender': '1', 'Final_Time_Hund': 9559}, {'Horse Name': 'Road Test', 'Final_Time': '1:36.12', 'Gender': '1', 'Final_Time_Hund': 9612}, {'Horse Name': 'Hotsy Dotsy', 'Fina

In [25]:
dffinaltime3 = pd.DataFrame.from_dict(timesdic)

#dfpract = dfpract.transpose()
dffinaltime3

,Final_Time,Final_Time_Hund,Gender,Horse Name
0,1:40.25,10025,1,Dhafeera
1,1:40.46,10046,1,Jazaalah
2,1:40.58,10058,1,Meal Ticket
3,1:40.84,10084,1,You Missed It
4,1:40.89,10089,1,Blossom Trail Miss
5,1:40.99,10099,1,Chalky
6,1:41.75,10175,1,Beauty Divine
7,1:35.18,9518,1,Subic Bay
8,1:35.59,9559,1,Oh Scatty Oh
9,1:36.12,9612,1,Road Test


## Read df_stats3 to act on it

In [55]:
df_stats3 = pd.read_csv('stats_csv_3')

In [56]:
df_stats3 = df_stats3.drop('Unnamed: 0', axis=1)

In [57]:
#For Reference
Race_one = ['Dhafeera','Jazaalah','Meal Ticket','You Missed It','Blossom Trail Miss','Chalky','Beauty Divine']
#Maiden Special Weight
Race_five = ['Subic Bay','Oh Scatty Oh','Road Test','Hotsy Dotsy','Paddy Jean','Khaleesi','Twirling Apples',
             'Lil Bit Dangerous','Pirate Flag']
#Claiming

## Merge final time and df stats to clean

In [58]:
finalmerged3 = pd.merge(dffinaltime3,df_stats3,on='Horse Name', how = 'left')
finalmerged3

,Final_Time,Final_Time_Hund,Gender,Horse Name,Firsts,Seconds,Starts,Thirds
0,1:40.25,10025,1,Dhafeera,1,2,6,0
1,1:40.46,10046,1,Jazaalah,0,1,5,2
2,1:40.58,10058,1,Meal Ticket,0,2,7,1
3,1:40.84,10084,1,You Missed It,0,5,8,1
4,1:40.89,10089,1,Blossom Trail Miss,0,0,4,1
5,1:40.99,10099,1,Chalky,0,2,6,1
6,1:41.75,10175,1,Beauty Divine,0,0,4,0
7,1:35.18,9518,1,Subic Bay,2,3,10,1
8,1:35.59,9559,1,Oh Scatty Oh,1,4,15,1
9,1:36.12,9612,1,Road Test,1,5,16,4


In [59]:
finalmerged3['Starts'] = finalmerged3['Starts'] - 1
finalmerged3

,Final_Time,Final_Time_Hund,Gender,Horse Name,Firsts,Seconds,Starts,Thirds
0,1:40.25,10025,1,Dhafeera,1,2,5,0
1,1:40.46,10046,1,Jazaalah,0,1,4,2
2,1:40.58,10058,1,Meal Ticket,0,2,6,1
3,1:40.84,10084,1,You Missed It,0,5,7,1
4,1:40.89,10089,1,Blossom Trail Miss,0,0,3,1
5,1:40.99,10099,1,Chalky,0,2,5,1
6,1:41.75,10175,1,Beauty Divine,0,0,3,0
7,1:35.18,9518,1,Subic Bay,2,3,9,1
8,1:35.59,9559,1,Oh Scatty Oh,1,4,14,1
9,1:36.12,9612,1,Road Test,1,5,15,4


In [60]:
#Setting index so we can subtract 1st, 2nd, 3rd place counts for that day's race. 
finalmerged3 = ( finalmerged3.set_index(['Horse Name']))
finalmerged3

,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds
Horse Name,,,,,,,
Dhafeera,1:40.25,10025,1,1,2,5,0
Jazaalah,1:40.46,10046,1,0,1,4,2
Meal Ticket,1:40.58,10058,1,0,2,6,1
You Missed It,1:40.84,10084,1,0,5,7,1
Blossom Trail Miss,1:40.89,10089,1,0,0,3,1
Chalky,1:40.99,10099,1,0,2,5,1
Beauty Divine,1:41.75,10175,1,0,0,3,0
Subic Bay,1:35.18,9518,1,2,3,9,1
Oh Scatty Oh,1:35.59,9559,1,1,4,14,1


In [67]:
finalmerged3.loc[['Dhafeera','Subic Bay'],['Firsts']] = finalmerged3.loc[['Dhafeera','Subic Bay'],['Firsts']]-1

In [68]:
finalmerged3.loc[['Jazaalah','Oh Scatty Oh'], ['Seconds']] = finalmerged3.loc[['Jazaalah','Oh Scatty Oh'], ['Seconds']]-1

In [69]:
finalmerged3.loc[['Meal Ticket','Road Test'], ['Thirds']] = finalmerged3.loc[['Meal Ticket','Road Test'], ['Thirds']]-1

In [70]:
finalmerged3

,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds
Horse Name,,,,,,,
Dhafeera,1:40.25,10025,1,0,2,5,0
Jazaalah,1:40.46,10046,1,0,0,4,2
Meal Ticket,1:40.58,10058,1,0,2,6,0
You Missed It,1:40.84,10084,1,0,5,7,1
Blossom Trail Miss,1:40.89,10089,1,0,0,3,1
Chalky,1:40.99,10099,1,0,2,5,1
Beauty Divine,1:41.75,10175,1,0,0,3,0
Subic Bay,1:35.18,9518,1,1,3,9,1
Oh Scatty Oh,1:35.59,9559,1,1,3,14,1


In [71]:
finalmerged3.to_csv('final_merged3')
